# 第7回 その3: エコーキャンセリング
NLMSアルゴリズムによるエコーキャンセリングのデモンストレーションです。


## ステップ0: Google Driveのマウントと作業フォルダへの移動  
Google Drive に配置したデータを読み込むための準備です。  
詳細については第二回の 02_01_graph.ipynb を参照してください。  

ここでは"マイドライブ/情報管理/07"を作業フォルダとします。 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# フォルダの移動には"%cd"を使用します。
# 作業フォルダへ移動
%cd /content/drive/'My Drive'/情報管理/07/
# 現在のフォルダの中身を表示
%ls

`echo_mixed_input.wav`と`reference.wav`というデータが表示されていることを確認してください。

必要なライブラリをインポートしておきます。

In [ ]:
import wave as wave
import numpy as np
import matplotlib.pyplot as plt

## ステップ1: マイク収録信号（音声＋雑音）とスピーカ再生音データの読み込み  
まずマイクで収録された音声信号 `echo_mixed_input.wav` を読み込みます。  

In [ ]:
wav_file = 'echo_mixed_input.wav'
with wave.open(wav_file, 'rb') as wav:
  # サンプリング周波数 [Hz] を取得
  sampling_frequency = wav.getframerate()
  # その他の情報（ファイルサイズ等，書き込み時に必要）を取得
  wav_params = wav.getparams()
  # wavデータを読み込む
  input_data = wav.readframes(wav.getnframes())
  # 読み込んだデータはバイナリ値(16bit short型)なので，numpy array型の数値ベクトル(32bit float型)に変換する
  input_data = np.frombuffer(input_data, dtype=np.int16).astype(np.float32)

# データをプロット
plt.figure(figsize=(10,5))
x = np.arange(np.size(input_data)) / sampling_frequency # 横軸データ
plt.plot(x, input_data)
plt.xlabel('Time [second]')
plt.ylabel('Amplitude')
plt.show()

import IPython.display
IPython.display.display(IPython.display.Audio('echo_mixed_input.wav'))

続いて，スピーカから再生させる音楽データ（ここでは**参照信号**(**reference**)と呼びます）を読み込みます。  
（再生している音楽は ヴィヴァルディ「四季」より「冬」です。）

In [ ]:
ref_file = 'reference.wav'
with wave.open(ref_file, 'rb') as ref:
  # （サンプリング周波数やその他の情報の取得は省略する。）
  # wavデータを読み込む
  reference_data = ref.readframes(ref.getnframes())
  # 読み込んだデータはバイナリ値(16bit short型)なので，numpy array型の数値ベクトル(32bit float型)に変換する
  reference_data = np.frombuffer(reference_data, dtype=np.int16).astype(np.float32)

# データをプロット
plt.figure(figsize=(10,5))
x = np.arange(np.size(reference_data)) / sampling_frequency # 横軸データ
plt.plot(x, reference_data)
plt.xlabel('Time [second]')
plt.ylabel('Amplitude')
plt.show()

IPython.display.display(IPython.display.Audio('reference.wav'))

## ステップ2: NLMSアルゴリズム
NLMSアルゴリズムの式を以下にまとめます。  
$y[n] = \sum_{k=0}^{K-1}\hat{h}_{\rm old}[k]x[n-k]$  
$e[n] = z[n] - y[n]$  
$\hat{h}_{\rm new} = \hat{h}_{\rm old} + \frac{\mu}{\sum_{k=0}^{K-1}(x[n-k])^{2}+\epsilon}e[n]x[n-k]$  

$n$，$k$は時刻を表す記号  
$x[n]$はスピーカから再生する信号（参照信号，`reference_data`）  
$\hat{h}_{\rm old}[k]$，$\hat{h}_{\rm new}[k]$は推定したインパルス応答（`estimated_rir`，長さ $K$）。oldは更新前，newは更新後。  
$y[n]$は推定したマイク入力音（`estimated_echo`）  
$z[n]$は実際のマイク入力音（`input_data`）  
$e[n]$はエコーキャンセリングによる消し残り（`error`，`output_data`）  
$\mu$はNLMSアルゴリズムにおける学習率（`myu`）  
$\epsilon$はゼロ割を防ぐための小さい値（1e-10）


事前に設定すべきパラメータ（機械学習分野において，ハイパーパラメータと呼ばれます）を設定します。

In [ ]:
# 想定するインパルス応答の長さ
K = 1024

# インパルス応答の学習率  
myu = 1.0

# インパルス応答の学習を止める時刻
stop_update_time = 10.0

# ゼロ割防止のための小さい値
eps = 1e-10

`stop_update_time = 10.0` はインパルス応答の学習を止める時刻です。  
言い換えると，この設定では，最初の10秒間の音声データを使ってインパルス応答の学習を行います。  

以下に，NLMSアルゴリズムを実装します。

In [ ]:
# マイク収録音(input_data)の長さ
N = np.size(input_data)
# 推定されたインパルス応答
estimated_rir = np.zeros(K)
# エコーキャンセリング後の信号
output_data = np.zeros(N)

for n in range(N):
  if n-K+1 < 0:
    # n-K+1<0の場合，以降の処理においてベクトルの
    # インデクスが負になるため，この場合は畳み込みをしない。
    output_data[n] = input_data[n]
  else:
    #
    # NLMSアルゴリズムによるエコーキャンセリングを実行する。
    #

    # 現在の推定インパルス応答と参照信号の畳み込みを行い，
    # スピーカの収録信号（エコー）を推定する。
    # 畳み込みについては07_02_convolve_rir.ipynbを参照
    reference_part = reference_data[n-K+1:n+1]
    reference_rev = np.flip(reference_part)
    estimated_echo = np.dot(estimated_rir, reference_rev)

    # 入力信号からエコー信号を引くことで、エコーを除去する。
    # このとき、減算結果がエコーの消し残り(error)である。
    output_data[n] = input_data[n] - estimated_echo
    error = output_data[n]

    # 正規化項(参照信号のパワー)を求める。  
    norm = np.sum(reference_rev**2) + eps

    # インパルス応答の更新を止める時間(stop_update_time * sampling_rate)に
    # 達していない場合はインパルス応答の推定値を更新する
    if n < stop_update_time * sampling_frequency:
      estimated_rir += (myu / norm) * error * reference_rev


実行結果を出力します。  

In [ ]:
out_file = 'output.wav'
with wave.open(out_file, 'wb') as out:
  # 音声データの情報（wav_params）をセット
  out.setparams(wav_params)
  # numpy array型(32bit float)のデータをバイナリデータ（16bit short）に変換
  out_binary_data = output_data.astype(np.int16).tobytes()
  # データを書き込む
  out.writeframes(out_binary_data)

# データをプロット
plt.figure(figsize=(10,5))
x = np.arange(np.size(output_data)) / sampling_frequency # 横軸データ
plt.plot(x, output_data)
plt.xlabel('Time [second]')
plt.ylabel('Amplitude')
plt.show()

import IPython.display
IPython.display.display(IPython.display.Audio('output.wav'))

音楽が抑圧されて，音声が聞き取りやすくなっていることが分かります。
最初の時刻は消し残りが大きく，時間がたつにつれて雑音除去の効果が強まっていくのは，最初の時刻ではインパルス応答が正確に推定できておらず，勾配降下法によって更新が進むにつれてインパルス応答の推定が正確になっているためです。

推定（学習）したインパルス応答を出力します。

In [ ]:
# データをプロット
plt.figure(figsize=(10,5))
x_r = np.arange(np.size(estimated_rir)) / sampling_frequency # 横軸データ
plt.plot(x_r, estimated_rir)
plt.xlabel('Time [second]')
plt.ylabel('Impulse response')
plt.show()

以上がNLMSエコーキャンセリングのデモンストレーションです。  
今回はインパルス応答が時間変化せず，かつ最初の10秒間において音声が入っていないという理想的な環境をシミュレーションしたデータを用いたため，きれいに雑音除去ができました。  
しかし現実世界はインパルス応答が部屋の環境変化（部屋に置いてある物体が移動，空気の流れ，気温，など）によって時々刻々と変化するなど，今回のシミュレーションデータとは異なる点が沢山あるため，エコーキャンセリングは容易ではありません。  
レポート課題でこの難しさについて触れてみてください。